In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import random

from baseline import exhaustive, random_placement, heft, random_op_greedy_dev, random_op_est_dev
from env.utils import generate_program, generate_network
from env.network import StarNetwork
from env.program import Program
from env.latency import evaluate

from placement_rl.placement_env import PlacementEnv
from placement_rl.placement_agent import PlacementAgent

import os
import pickle

import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Using backend: pytorch


In [2]:
n_devices = [10, 20, 30]
n_operators = [10, 20, 30]
networks = {n: StarNetwork(*generate_network(n, seed=0)) for n in n_devices}
programs = {}
for n in n_devices:
    programs[n] = {}
    for m in n_operators:
        programs[n][m] = []
        for seed in range(5):
            DAG, constraints = generate_program(m, n, seed=seed)
            programs[n][m].append(Program(DAG, constraints))

In [ ]:
results = []
noises = [0, 0.01, 0.05, 0.1]
for ndev in [10]:
    network = networks[ndev]
    for nop in n_operators:
        for seed, program in enumerate(programs[ndev][nop]):
            for noise in [0]:
                s = time.time()
                heft_map, heft_lat = heft(program, network, noise)
                t = time.time() - s
                results.append([ndev, nop, seed, noise, t, heft_map, heft_lat])
                print(results[-1])
                

[10, 10, 0, 0, 0.1030280590057373, [3, 2, 2, 9, 9, 4, 4, 4, 8, 7], tensor(552.2978, dtype=torch.float64)]
[10, 10, 1, 0, 0.07746291160583496, [2, 8, 8, 2, 8, 1, 9, 8, 2, 6], tensor(487.3011, dtype=torch.float64)]
[10, 10, 2, 0, 0.062454938888549805, [5, 8, 8, 8, 2, 8, 5, 5, 2, 7], tensor(791.6463, dtype=torch.float64)]
[10, 10, 3, 0, 0.06872916221618652, [0, 0, 0, 2, 2, 1, 8, 2, 8, 4], tensor(473.6158, dtype=torch.float64)]
[10, 10, 4, 0, 0.07308793067932129, [8, 2, 8, 1, 8, 2, 8, 8, 8, 4], tensor(443.2503, dtype=torch.float64)]
[10, 20, 0, 0, 8.78424596786499, [9, 0, 2, 2, 2, 8, 2, 8, 9, 9, 9, 8, 3, 2, 0, 1, 9, 9, 0, 7], tensor(1287.5770, dtype=torch.float64)]
[10, 20, 1, 0, 17.13996911048889, [6, 2, 2, 2, 2, 8, 2, 2, 2, 2, 9, 0, 0, 0, 0, 9, 2, 2, 6, 1], tensor(826.2315, dtype=torch.float64)]
[10, 20, 2, 0, 22.05184578895569, [2, 9, 9, 2, 9, 9, 9, 8, 9, 9, 9, 9, 9, 8, 2, 2, 8, 9, 2, 8], tensor(841.7935, dtype=torch.float64)]
[10, 20, 3, 0, 31.923469066619873, [8, 8, 8, 8, 2, 1, 8, 1, 